In [1]:
import os, math
import _pickle as pickle
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [2]:
dt = np.loadtxt("parts.csv", delimiter=",")

In [3]:
dt.shape
## The date range
time_list = np.array(range(51))
month_list = time_list % 12
year_list = (time_list /12).astype(int)

In [4]:
parts_index = list(range(1046))
sliding_window_dis = 4;# can change to 24
sample_len = 24; #input 12,output 12
input_len = 12;
output_len = 12;
total_n = 7
test_n = 1
train_n = 4

In [5]:
trainX_list = []; trainX2_list = []; trainY_list = [];trainY2_list = []
testX_list = [];  testX2_list = [];  testY_list = [];testY2_list = []
#for part in parts_index:
for part in parts_index:
    sub_series = dt[part,:]

    trainX = np.zeros(shape=(train_n, input_len))       ## The input series
    trainY = np.zeros(shape=(train_n, output_len))      ## The output series  

    testX  = np.zeros(shape=(test_n, input_len))        ## The input series of testing dataset
    testY = np.zeros(shape=(test_n, output_len))        ## The output series of testing dataset

    covariate_num = 3   # other features covariate_num: parts_id, n_year, n_month
    trainX2 = np.zeros(shape=(train_n, input_len, covariate_num))
    trainY2 = np.zeros(shape=(train_n, output_len, covariate_num))
    testX2 = np.zeros(shape=(test_n, input_len, covariate_num))
    testY2 = np.zeros(shape=(test_n, output_len, covariate_num))

    ts_len = sub_series.shape[0]
    start_index = ts_len-sample_len

    for i in range(total_n):
        ### The ts sequences
        series_x,series_y = sub_series[start_index:start_index+input_len],sub_series[start_index+input_len:start_index+sample_len]
        ### The covariate
        parts_X = np.repeat(part, input_len)
        parts_Y = np.repeat(part, output_len)
        ### The time-index covariates
        year_index_x, year_index_y = year_list[start_index:start_index+input_len], year_list[start_index+input_len:start_index+sample_len]
        month_index_x, month_index_y = month_list[start_index:start_index+input_len], month_list[start_index+input_len:start_index+sample_len]
        covariate_X = np.c_[parts_X, year_index_x, month_index_x]
        covariate_Y = np.c_[parts_Y, year_index_y, month_index_y]
    
        if(i==0):
            testX[i] = series_x
            testY[i] = series_y
            testX2[i,:,:] = covariate_X
            testY2[i,:,:] = covariate_Y
        if(i>2):
            trainX[i-3] = series_x
            trainY[i-3] = series_y
            trainX2[i-3] = covariate_X
            trainY2[i-3] = covariate_Y
        # update the start_index
        start_index = start_index - sliding_window_dis
        
    testX_list.append(testX)
    testX2_list.append(testX2)
    testY_list.append(testY)
    testY2_list.append(testY2)
    
    trainX_list.append(trainX)
    trainX2_list.append(trainX2)
    trainY_list.append(trainY)
    trainY2_list.append(trainY2)

In [6]:
testX_dt = np.vstack(testX_list)
testY_dt = np.vstack(testY_list)
testX2_dt = np.vstack(testX2_list)
testY2_dt = np.vstack(testY2_list)

trainX_dt = np.vstack(trainX_list)
trainY_dt = np.vstack(trainY_list)
trainX2_dt = np.vstack(trainX2_list)
trainY2_dt = np.vstack(trainY2_list)

In [7]:
### Save the data
with open('feature_prepare.pkl', 'wb') as f:
    pickle.dump([trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt], f, -1)